In [2]:
%run packages.py

/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
!pip install yfinance

  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.3.0-cp39-cp39-macosx_10_9_universal2.whl.metadata (3.8 kB)
  Using cached frozendict-2.4.6-cp39-cp39-macosx_11_0_arm64.whl.metadata (23 kB)
  Using cached peewee-3.17.8-cp39-cp39-macosx_10_9_universal2.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached frozendict-2.4.6-cp39-cp39-macosx_11_0_arm64.whl (37 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.3.0-cp39-cp39-macosx_10_9_universal2.whl (8.1 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)


In [5]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2019-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]


In [7]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [8]:
start_date = '2018-01-01'
end_date = '2019-12-31'
period = '1d'
colone = ['Close']


In [9]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [10]:
raw_data = Enginner.RawData()
raw_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'AOS': Price           Close
 Ticker            AOS
 Date                 
 2018-01-02  54.426792
 2018-01-03  54.754082
 2018-01-04  55.010609
 2018-01-05  55.718258
 2018-01-08  55.983620
 ...               ...
 2019-12-23  43.067322
 2019-12-24  43.396221
 2019-12-26  43.615490
 2019-12-27  43.679436
 2019-12-30  43.460167
 
 [502 rows x 1 columns],
 'AMD': Price           Close
 Ticker            AMD
 Date                 
 2018-01-02  10.980000
 2018-01-03  11.550000
 2018-01-04  12.120000
 2018-01-05  11.880000
 2018-01-08  12.280000
 ...               ...
 2019-12-23  45.459999
 2019-12-24  46.540001
 2019-12-26  46.630001
 2019-12-27  46.180000
 2019-12-30  45.520000
 
 [502 rows x 1 columns],
 'ALB': Price            Close
 Ticker             ALB
 Date                  
 2018-01-02  120.240570
 2018-01-03  120.478249
 2018-01-04  119.042976
 2018-01-05  120.414253
 2018-01-08  125.652412
 ...                ...
 2019-12-23   68.524284
 2019-12-24   68.420212
 2019-12-26   68.

In [15]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)

/Users/mcbookairdebat/Desktop/Projet/project/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Projet/project/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Projet/project/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Projet/project/Pair_trading_strategy/Pai

In [12]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)

In [13]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()


/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_70146/64972235.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()


In [12]:
Metric = Pair_Selection(dataframe,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list)

In [13]:
distance = Metric.compute_euclidian_distance()


In [14]:

selected_pairs = Metric.paire_selection(distance,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,pairs=pairs)

In [31]:
data1 = dataframe_normalized.copy()
data2 = raw_dataframe.copy()
# The goal would be to stock every small data frame in a dictionnay to help doing the steps for the reste 
dataframes_dictionnary_norm = {}
dataframes_dictionnary_raw ={}
for sector in sector_list: 
    for value in selected_pairs[sector]:
        
        dataframes_dictionnary_norm[value] = data1[list(value)]
        #dataframes_dictionnary_raw[value] = data2[list(value)]

In [34]:
from Spread import Spread


In [35]:
compute_spread = Spread(dataframes_dictionnary_norm)
x = compute_spread.dollar_neutral_spread()

/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]]/values[key[1]])
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:15: SettingWithCo

In [36]:
x

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.600364  0.635466    2.160849
 2019-12-24  0.588530  0.622155    1.850249
 2019-12-26  0.615840  0.645341    1.841183
 2019-12-27  0.609012  0.631601    1.788587
 2019-12-30  0.583068  0.603263    1.470415
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314614         NaN
 2018-01-08  0.174384  0.332054         NaN
 ...              ...       ...         ...
 2019-12-23

In [33]:
for key,values in dataframes_dictionnary_norm.items():
    values['Delta'] = np.abs(values[key[0]] - values[key[1]])
    values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
    values.drop(['Delta'], axis = 1, inplace=True )

dataframes_dictionnary_norm


/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]] - values[key[1]])
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:4: Se

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.600364  0.635466   -2.613264
 2019-12-24  0.588530  0.622155   -1.971344
 2019-12-26  0.615840  0.645341   -1.857730
 2019-12-27  0.609012  0.631601   -1.897010
 2019-12-30  0.583068  0.603263   -1.593062
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314614         NaN
 2018-01-08  0.174384  0.332054         NaN
 ...              ...       ...         ...
 2019-12-23